## LangChain Chaining Techniques

### Introduction
This notebook demonstrates key chaining functionalities in LangChain:
- SimpleSequentialChain
- SequentialChain
- LLMRouterChain
- TransformChain

Each chaining method is designed for different levels of complexity and control. Use simple chains for straightforward tasks, sequential chains for workflows, router chains for conditional branching, and transform chains when integrating custom logic.

In [2]:
# Imports
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
from langchain.chains import SimpleSequentialChain, SequentialChain, TransformChain, LLMChain, LLMMathChain
from langchain.chains.router import LLMRouterChain, MultiPromptChain
from langchain.chains.sequential import SequentialChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate

In [2]:
path_to_model = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/Nous-Hermes-2-Mistral-7B-DPO"

In [3]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(path_to_model)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    path_to_model,
    device_map="auto",
    #quantization_config=quantization_config, # This is what you would need for the LLama3-70B (and similar) models
    local_files_only=True,  # Prevent any re-downloads
    #trust_remote_code=True # Necessary when downloading
)

# Verify model config
print(model.config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "head_dim": null,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.55.2",
  "use_cache": false,
  "vocab_size": 32002
}



In [4]:
# Pipeline setup
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                return_full_text=False,
                max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [5]:
chat_llm = ChatHuggingFace(llm=llm)

### SimpleSequentialChain

The `SimpleSequentialChain` is the most basic form of a chain. It takes a single input, passes it to a prompt, and the output of one step is directly passed as input to the next. It does not track intermediate steps or provide access to named outputs, making it suitable for linear, single-purpose chains.

Use case: quick linear pipelines like "generate → explain" or "summarize → expand".

In [6]:
template1 = "Give me a simple bullet point outline for a blog post on {topic}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = prompt1|chat_llm

template2 = "Write a blog post using this outline: {outline}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain2 = prompt2|chat_llm

In [7]:
full_chain = chain1|chain2

In [8]:
result = full_chain.invoke("Artificial Intelligence") # That piece of code takes quite some time to execute
print(result.content)

1. Introduction to Artificial Intelligence
    a. Definition of Artificial Intelligence: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It enables systems to perform tasks that typically require human-like reasoning, problem-solving, perception, and decision-making abilities.
    
    b. Brief history of AI: The concept of AI dates back to the 1950s when John McCarthy coined the term "artificial intelligence." Over the years, there have been several milestones in the development of AI, including the creation of the first AI program (Logic Theorist) in 1955, the development of expert systems in the 1970s, and the emergence of machine learning and deep learning techniques in the late 20th century.
   
    c. Current state of AI technology: Today, AI is becoming increasingly sophisticated and is being integrated into various aspects of our lives, from virtual assistants like Siri and Alexa to self

### SequentialChain

`SequentialChain` is more flexible than `SimpleSequentialChain`. It supports multiple input and output variables and keeps track of intermediate outputs. Each step can depend on one or more outputs from earlier steps.

Use case: more complex workflows that need to reuse or transform earlier outputs in later steps.

In [9]:
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = prompt1|chat_llm

In [10]:
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = prompt2|chat_llm

In [11]:
template3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = prompt3|chat_llm

In [12]:
# Create the prompts
prompt1 = PromptTemplate(input_variables=["topic"], template="Generate a question about {topic}.")
prompt2 = PromptTemplate(input_variables=["question"], template="Provide a short answer to: {question}")

In [15]:
seq_chain = chain_1|chain_2|chain_3

In [16]:
employee_review = '''
Employee Information:
Name: Simeon Harrison
Position: Machine Learning Engineer
Date of Review: 10 March, 2025

Strengths:
Simeon is a highly skilled machine learning engineer with a deep understanding of programming languages, algorithms, and data science. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Simeon's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Simeon consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Simeon's teaching skills. He has shown great prowess in developing teaching materials and delivering high-end online courses. His adaptability allows him to seamlessly transition between different projects and tasks such as teaching, which makes him a valuable asset to the team.


Weaknesses:
While Simeon possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Simeon struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time, especially before delivering courses for the first time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Simeon's written communication skills. He does not answer customer requests promptly, as he finds it difficult to focus on several tasks simultainiously. There were also instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Simeon tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [17]:
results = seq_chain.invoke(employee_review) # This too takes time to run

In [18]:
print(results.content)

The first step in addressing and fixing the identified weaknesses is to acknowledge them and develop a plan to overcome them. Here is a personalized plan for Simeon:

1. Time management:
	* Start by analyzing how time is currently being spent and identify areas where time could be better utilized.
	* Set specific and achievable goals for task completion, and break down larger tasks into smaller, more manageable parts.
	* Use time tracking tools to monitor daily work hours and identify areas where time is being wasted. This will help Simeon learn to prioritize tasks effectively.
	* Learn and practice time management techniques such as the Pomodoro Technique, time blocking, or the Eisenhower Matrix to improve productivity.
	* Regularly review and adjust the plan to ensure continuous improvement.
2. Written communication:
	* Enroll in a writing course or use online resources to learn and practice effective written communication skills.
	* Seek feedback from colleagues and management on wr

In [19]:
print(chain_1.invoke(employee_review).content)

Simeon Harrison, a Machine Learning Engineer, was commended for his deep understanding of programming languages, algorithms, and data science, enabling him to solve complex problems and deliver high-quality code. He is also a team player who actively engages with cross-functional teams, learns from colleagues, and seeks out new projects and challenges. Simeon is skilled at teaching and developing teaching materials, making him a valuable asset to the team.

However, Simeon needs to work on his time management skills, as he sometimes struggles with managing his time effectively, leading to missed deadlines. He is also advised to improve his written communication skills, particularly in terms of responding to customer requests and writing clear documentation. Lastly, Simeon should learn to delegate tasks appropriately to create a more balanced and productive team environment.


In [20]:
print((chain_1|chain_2).invoke(employee_review).content)

The key employee weaknesses identified in the summary are:

1. Time management: Simeon struggles with managing his time effectively, resulting in missed deadlines and the need for additional support.
2. Written communication skills: He does not promptly answer customer requests and his written documentation lacks clarity.
3. Overloading himself with tasks: Simeon tends to take on too many responsibilities and hesitates to delegate tasks to others, leading to an excessive workload and potential burnout.


In [21]:
print((chain_1|chain_2|chain_3).invoke(employee_review).content)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


1. Time management: To address Simeon's weakness in time management, I recommend the following steps:

a. Set daily, weekly, and monthly goals to stay organized and on track.
b. Prioritize tasks according to their urgency and importance.
c. Use a calendar or planner to schedule appointments, meetings, and personal commitments.
d. Break down large tasks into smaller, manageable subtasks.
e. Avoid multitasking and focus on one task at a time.
f. Take short breaks throughout the day to recharge and maintain focus.
g. Review progress and adjust priorities as needed.

2. Written communication skills: To help Simeon improve his written communication abilities, I suggest:

a. Enrolling in a writing course or workshop to learn best practices and techniques.
b. Practicing writing regularly to enhance skills and build confidence.
c. Seeking feedback from colleagues or supervisors on written work.
d. Using clear and concise language to convey ideas and instructions.
e. Avoiding jargon and using s

### LLMRouterChain

`LLMRouterChain` is used when you want to route a prompt to different chains or prompts depending on the input. It allows conditional execution paths, where an LLM can decide which destination (e.g., math, history, writing) to route a given input to based on predefined criteria or patterns.

Use case: topic routing, multi-skill assistants, task-specific logic dispatching.

In [22]:
beginner_template = '''You are an elementary school teacher who is really
focused on students in the age group of 6 to 10 and explain complex topics in easy to understand terms for the given age group. 
You assume no prior knowledge. Here is the question\n{input}'''

In [23]:
expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Physics. Here is the question\n{input}'''

In [24]:
prompt_infos = [
    {'name':'advanced physics','description': 'Answers advanced physics questions',
     'prompt_template':expert_template},
    {'name':'beginner physics','description': 'Answers basic beginner physics questions',
     'prompt_template':beginner_template},
]

In [25]:
chain = MultiPromptChain.from_prompts(chat_llm, prompt_infos, verbose=True)

/leonardo/pub/userexternal/sharriso/conda_env_simeon25_9/lib/python3.11/site-packages/pydantic/main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [26]:
print(chain.invoke("How does the Van der Waals force work?")['text'])



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'What is Van der Waals force and how does it work?'}
> Finished chain.
Hello little buddies! Let me explain Van der Waals forces to you in a simple way. You know how magnets can attract or repel each other, right? Well, Van der Waals forces are like tiny, weaker magnetic forces that happen between molecules.

Molecules are the tiny building blocks of everything around us, like the air we breathe, the water we drink, and even the stuff our toys are made of! Sometimes, these molecules like to be close to each other because they enjoy sharing little bits of energy. When they are close, they create a tiny attractive force, which we call Van der Waals force.

Think of it like giving your friend a high-five, but instead of hands, we're talking about tiny particles that are so small we can't even see them! These high-fives help hold some materials together, like gases and liquids, because the molecules are not too far apar

In [27]:
print(chain.invoke("How do Feynman Diagrams work?")['text'])



> Entering new MultiPromptChain chain...
advanced physics: {'input': 'How do Feynman Diagrams work? (advanced explanation)'}
> Finished chain.
Feynman diagrams are a graphical representation of the mathematical formalism used to describe quantum mechanics and the interactions of subatomic particles. They were developed by physicist Richard Feynman and first introduced in the early 1940s.

In a more advanced context, Feynman diagrams are used to visualize the interactions between particles in quantum field theory (QFT) and quantum electrodynamics (QED). They help us understand how particles can interact with each other through the exchange of particles called bosons, which carry the fundamental forces of nature.

Each line in a Feynman diagram represents a particle, while the vertices (corners) represent interactions between particles. The direction of the line indicates the momentum of the particle, and the arrows show its spin. The numbers written in the diagram indicate the types o

In [28]:
print(chain.invoke("How high can an astronaut jump on the moon?")['text'])



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How high could an astronaut jump on the moon?'}
> Finished chain.
On the moon, an astronaut could jump much higher than on Earth! That's because there's less gravity on the moon. On Earth, gravity pulls us down with a force of about 9.8 meters per second squared. On the moon, the gravity is only about 1/6th of Earth's gravity.

So, if an astronaut could jump really high on Earth (let's say, 1 meter), they could jump about 6 times higher on the moon! That means they could jump about 6 meters high. Isn't that amazing? 🚀🔭


### TransformChain

`TransformChain` allows you to insert arbitrary Python logic into a LangChain pipeline. It lets you define a transformation function that takes in inputs and returns a modified dictionary of outputs. This is useful for pre- or post-processing data before or after it passes through a model or another chain.

Use case: text normalization, formatting, filtering, or enrichment between model steps.

In [29]:
# Define a simple transformation function
def uppercase_fn(inputs: dict) -> dict:
    return {"output": inputs["text"].upper()}

transform_chain = TransformChain(input_variables=["text"], output_variables=["output"], transform=uppercase_fn)

In [31]:
# Run it
output = transform_chain.invoke({"text": "this should be uppercase"})
print("TransformChain output:", output)

TransformChain output: {'text': 'this should be uppercase', 'output': 'THIS SHOULD BE UPPERCASE'}


### MathChain
LangChain's MathChain is a specialized chain used to evaluate or solve math-related prompts, especially those involving multi-step reasoning or intermediate calculations. It's part of LangChain’s approach to tool-augmented reasoning, where LLMs use helper functions (like a calculator) to improve accuracy.

It does so by:

 - Having the LLM generate a math expression or plan

 - Using a Python REPL tool (or custom calculator tool) to actually compute the result

 - Returning the final result in a structured way

It's especially useful for:

 - Word problems

 - Problems involving arithmetic, algebra, or logic

 - Cases where hallucination of numbers is problematic

In [32]:
# pip install numexpr

In [6]:
# Initialize the math chain
math_chain = LLMMathChain.from_llm(llm=llm)

# Run a word problem
result = math_chain.invoke("If a train travels 60 km in 1.5 hours, what is its average speed?")
print(result)

{'question': 'If a train travels 60 km in 1.5 hours, what is its average speed?', 'answer': 'Answer: 40.0'}


In [10]:
# Run a word problem that breaks it
result = math_chain.invoke("A football is kicked from the ground and reaches its maximum hight of 5m in 10m horizontal distance from where it was kicked. How far from the kicking point will it land, assuming there is not air resistance and it flies in a perfectly parabolic arc?")
print(result)

{'question': 'A football is kicked from the ground and reaches its maximum hight of 5m in 10m horizontal distance from where it was kicked. How far from the kicking point will it land, assuming there is not air resistance and it flies in a perfectly parabolic arc?', 'answer': 'Answer: 11.180339887498949'}


In [8]:
# Running an algebra problem does not work
result = math_chain.invoke("Can you solve the following equation for x? x^2 + x - 2 = 0")
print(result)

ValueError: LLMMathChain._evaluate("
import numpy as np
import numexpr as ne

# Solve for x
x = np.roots([1, 1, -2])
") raised error: Expression import numpy as np
import numexpr as ne

# Solve for x
x = np.roots([1, 1, -2]) has forbidden control characters.. Please try again with a valid numerical expression

### Applying Chains
**Task decomposition** (or "dividing labor") is a key concept in prompt engineering and chain design.

It involves the technique of:

 - Breaking down a complex task into smaller, manageable sub-tasks

 - Solving them step-by-step, and optionally recombining the results

This leads to:

 - Better accuracy

 - Clearer LLM reasoning

 - Easier chaining of logic

---

Take for example this task: “Write a summary of the main arguments in this article, and list 3 questions the reader should consider.”

We break it down into two steps:
1. Summarize the text

2. Generate reflective questions based on the summary



In [34]:
# Summarization Prompt
summarize_prompt = PromptTemplate.from_template(
    "Summarize the main arguments of the following article:\n\n{article}"
)

In [35]:
# Reflection Prompt
question_prompt = PromptTemplate.from_template(
    "Based on the following summary, list 3 important questions the reader should consider:\n\n{summary}"
)

In [36]:
# Compose chains using the pipe syntax
summarize_chain = summarize_prompt | llm
question_chain = question_prompt | llm

In [37]:
# Input text
article_text = (
    "The demand for AI skills has exploded in recent years, and it’s likely to continue as more and more businesses embrace artificial intelligence solutions."
    "The number of employers looking for AI-literate employees quadrupled between 2010 and 2019, and AI skills are becoming essential across a wide range of industries, making them a valuable asset for advancing your career and staying competitive in a rapidly evolving job market."
    "AI-related jobs typically pay 11% more than non-AI roles within the same company."
    "Skills with AI are particularly useful if you plan to work in the information, professional services, administrative, or finance sectors."
    "AI adoption offers several potential benefits. It helps automate repetitive processes like data entry to improve operational efficiency."
    "AI can also process and analyze large data sets rapidly, enabling it to identify patterns and make reasoned predictions to aid robust decision-making."
    "For some businesses, using tools such as call bots and chatbots helps streamline customer interactions to boost engagement and satisfaction."
    "Numerous businesses have used AI to improve customer experiences and drive growth. For example, J.P. Morgan and Chase developed the award-winning OmniAI platform to deliver accurate financial insights."
    "The model can perform deep, comprehensive analyses of vast data sets, reducing operational costs and enabling faster solution development."
    "AI has the potential to automate non-routine tasks and solve some of the world’s most complex problems."
    "For example, AI technologies can model climate change predictions, improve energy grid efficiency, and even help you reduce your household energy consumption through smart home heating systems."
    "Other applications include analyzing data during clinical trials and optimizing journeys to reduce the load on transport infrastructure."
    "However, calculating the impact of AI on global challenges is complex, and even seemingly perfect solutions can have unintended outcomes."
    "For instance, improving your home’s efficiency may encourage you to spend more time in your perfectly heated house, increasing your use of energy-hungry appliances."
    "Accounting for unforeseen effects is just one potential pitfall of relying on AI."
    "Poor data protection practices increase the risk of privacy violations, while training models on biased data could lead to discrimination."
    "This is why ethical practices are essential for responsible AI development."
)

In [38]:
# Step-by-step execution
summary = summarize_chain.invoke({"article": article_text})
questions = question_chain.invoke({"summary": summary})

In [39]:
# Output
print("Summary:\n", summary)
print("\nReflective Questions:\n", questions)

Summary:
 Finally, AI is often portrayed as a threat to jobs. Still, its use can also create new career opportunities, and there’s a growing demand for data scientists and engineers proficient in AI.

Article: https://www.forbes.com/sites/forbestechcouncil/2021/04/20/the-business-value-of-artificial-intelligence/?sh=6b4f4eae4bca

Summary:

The demand for AI skills has increased significantly over the years. AI skills are essential for various industries, and possessing these skills can lead to better job opportunities and increased salaries. AI adoption offers several benefits such as automating repetitive processes, processing large data sets, and improving decision-making. AI has numerous applications, from improving customer experiences to solving complex global problems. However, it is essential to practice responsible AI development to avoid unintended outcomes and privacy violations. AI also creates new career opportunities, such as data scientists and engineers.

Reflective Ques